# Clustering Techniques: Hierarchical and DBSCAN Clustering
 
**Course Title:** MSCS 634  
**Lab Assignment:** Lab 5 - Clustering Techniques  

This lab explores Hierarchical and DBSCAN clustering methods using the Wine dataset from `sklearn.datasets`. We'll visualize the clusters, evaluate their quality, and compare the strengths and limitations of each approach.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Step 1: Data Preparation and Exploration

In [ ]:
from sklearn.datasets import load_wine
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load the Wine dataset
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = wine.target  # We won't use this for unsupervised clustering but can use for evaluation

In [ ]:
# Examine the dataset
print(X.head())
print(X.info())
print(X.describe())

In [ ]:
# Standardize the dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## Step 2: Hierarchical Clustering

In [ ]:
# %% [code]
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage

# Agglomerative Clustering for different cluster sizes in one figure
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))
cluster_sizes = [2, 3, 4]

for idx, n_clusters in enumerate(cluster_sizes):
    hc = AgglomerativeClustering(n_clusters=n_clusters)
    labels = hc.fit_predict(X_scaled)

    ax = axes[idx]
    sns.scatterplot(x=X_scaled[:, 0], y=X_scaled[:, 1], hue=labels, palette='Set2', ax=ax, legend=False)
    ax.set_title(f'Hierarchical Clustering (n_clusters={n_clusters})')
    ax.set_xlabel(wine.feature_names[0])
    ax.set_ylabel(wine.feature_names[1])

plt.tight_layout()
plt.show()


In [ ]:
# Generate Dendrogram
linked = linkage(X_scaled, 'ward')
plt.figure(figsize=(10, 5))
dendrogram(linked, truncate_mode='level', p=5)
plt.title("Hierarchical Clustering Dendrogram")
plt.xlabel("Sample Index")
plt.ylabel("Distance")
plt.show()

## Step 3: DBSCAN Clustering

In [ ]:
# %% [code]
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, homogeneity_score, completeness_score

# Try different eps and min_samples
eps_values = [0.5, 1.0, 1.5]
min_samples_values = [3, 5, 10]

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(18, 12))
axes = axes.flatten()

plot_idx = 0  # Index for subplot

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        db_labels = dbscan.fit_predict(X_scaled)

        n_clusters = len(set(db_labels)) - (1 if -1 in db_labels else 0)
        n_noise = list(db_labels).count(-1)

        ax = axes[plot_idx]
        sns.scatterplot(x=X_scaled[:, 0], y=X_scaled[:, 1], hue=db_labels, palette='tab10', ax=ax, legend=False)
        ax.set_title(f'DBSCAN (eps={eps}, min_samples={min_samples})\nClusters: {n_clusters}, Noise: {n_noise}')
        ax.set_xlabel(wine.feature_names[0])
        ax.set_ylabel(wine.feature_names[1])

        if len(set(db_labels)) > 1 and len(X_scaled) > n_noise:
            sil_score = silhouette_score(X_scaled, db_labels)
            homo_score = homogeneity_score(y, db_labels)
            comp_score = completeness_score(y, db_labels)
            print(f"eps={eps}, min_samples={min_samples} => Clusters: {n_clusters}, Noise: {n_noise}")
            print(f"Silhouette Score: {sil_score:.4f}")
            print(f"Homogeneity Score: {homo_score:.4f}")
            print(f"Completeness Score: {comp_score:.4f}")
        else:
            print(f"eps={eps}, min_samples={min_samples} => Clustering not valid (too few clusters)")

        plot_idx += 1

plt.tight_layout()
plt.show()


## Step 4: Analysis and Insights

Hierarchical clustering provided clear cluster separation when using 2 or 3 clusters, with a dendrogram offering helpful visualization of the hierarchical relationships. The method performed well overall and required only minimal parameter tuning.

DBSCAN was more sensitive to the `eps` and `min_samples` values. When tuned properly, DBSCAN was effective at identifying clusters and outliers. However, it sometimes failed to form valid clusters if the parameters were not set well. Unlike hierarchical clustering, DBSCAN can identify noise points, which is a strength in datasets with outliers.

In terms of evaluation metrics, Hierarchical clustering generally achieved higher homogeneity and completeness, while DBSCAN performed well only under optimal parameter conditions.

Overall, Hierarchical clustering is easier to apply and visualize on clean datasets like Wine, while DBSCAN excels in detecting non-convex clusters and noise.

## Submission Instructions

- Create a public GitHub repository (e.g., `MSCS_634_Lab_5`)
- Include:
  - This notebook file (`.ipynb`)
  - A `README.md` file explaining:
    - The purpose of the lab
    - Summary of insights and visualizations
    - Any challenges or decisions made
- Submit the link to your GitHub repository